📝 **Author:** Amirhossein Heydari - 📧 **Email:** <amirhosseinheydari78@gmail.com> - 📍 **Origin:** [mr-pylin/media-processing-workshop](https://github.com/mr-pylin/media-processing-workshop)

---


**Table of contents**<a id='toc0_'></a>    
- [Dependencies](#toc1_)    
- [Load Images](#toc2_)    
- [Geometric Transformations](#toc3_)    
  - [Rigid Transformations (Isometric)](#toc3_1_)    
    - [Translation (Shifting)](#toc3_1_1_)    
      - [Manual](#toc3_1_1_1_)    
      - [Using OpenCV](#toc3_1_1_2_)    
      - [Using PIL](#toc3_1_1_3_)    
      - [Using scikit-image](#toc3_1_1_4_)    
    - [Rotation](#toc3_1_2_)    
      - [Using OpenCV](#toc3_1_2_1_)    
      - [Using PIL](#toc3_1_2_2_)    
    - [Reflection (Flipping)](#toc3_1_3_)    
      - [Using OpenCV](#toc3_1_3_1_)    
  - [Affine Transformations](#toc3_2_)    
    - [Scaling](#toc3_2_1_)    
      - [Using OpenCV](#toc3_2_1_1_)    
    - [Shear](#toc3_2_2_)    
      - [Using OpenCV](#toc3_2_2_1_)    
  - [Projective Transformations (Homography)](#toc3_3_)    
    - [Perspective](#toc3_3_1_)    
      - [Using OpenCV](#toc3_3_1_1_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Dependencies](#toc0_)


In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import skimage as ski
from numpy.typing import NDArray
from PIL import Image, ImageChops

# <a id='toc2_'></a>[Load Images](#toc0_)


In [ ]:
im_1 = cv2.imread("../assets/images/dip_3rd/CH02_Fig0222(b)(cameraman).tif", flags=cv2.IMREAD_GRAYSCALE)
im_2 = cv2.cvtColor(
    cv2.imread("../assets/images/dip_3rd/CH06_Fig0638(a)(lenna_RGB).tif"),
    cv2.COLOR_BGR2RGB,
)

In [ ]:
img_1 = Image.fromarray(im_1)
img_2 = Image.fromarray(im_2)

In [ ]:
# plot
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(8, 4), layout="compressed")
axs[0].imshow(im_1, cmap="gray")
axs[0].set_title("CH02_Fig0222(b)(cameraman).tif")
axs[0].axis("off")
axs[1].imshow(im_2)
axs[1].set_title("CH06_Fig0638(a)(lenna_RGB).tif")
axs[1].axis("off")
plt.show()

# <a id='toc3_'></a>[Geometric Transformations](#toc0_)

- Geometric transformations **modify** the **spatial relationships of pixels** in an image.
- They change an image’s **position**, **size**, or **shape** by mapping input coordinates to new locations.

🆚 **Forward vs. Inverse Mapping:**

To apply a geometric transformation, two methods exist:
- **Forward Mapping**
  - **Directly** maps each pixel from **input** to **output**.
  - **Problem**: Some output pixels may not be assigned a value, causing holes.

$$
\begin{bmatrix}
x' \\ y' \\ 1
\end{bmatrix}
= M \cdot
\begin{bmatrix}
x \\ y \\ 1
\end{bmatrix}
$$

- **Inverse Mapping (Preferred)**
  - Computes where each output pixel came from in the input image.
  - Uses interpolation (**bilinear** or **bicubic**) to fill gaps.

$$
\begin{bmatrix}
x \\ y \\ 1
\end{bmatrix}
= M^{-1} \cdot
\begin{bmatrix}
x' \\ y' \\ 1
\end{bmatrix}
$$

📝 **Docs**:

- Affine Transformations: [docs.opencv.org/master/d4/d61/tutorial_warp_affine.html](https://docs.opencv.org/master/d4/d61/tutorial_warp_affine.html)
- `cv2.warpAffine`: [docs.opencv.org/master/da/d54/group__imgproc__transform.html#ga0203d9ee5fcd28d40dbc4a1ea4451983](https://docs.opencv.org/master/da/d54/group__imgproc__transform.html#ga0203d9ee5fcd28d40dbc4a1ea4451983)
- `cv2.warpPerspective`: [docs.opencv.org/master/da/d54/group__imgproc__transform.html#gaf73673a7e8e18ec6963e3774e6a94b87](https://docs.opencv.org/master/da/d54/group__imgproc__transform.html#gaf73673a7e8e18ec6963e3774e6a94b87)
- `PIL.ImageChops.offset`: [pillow.readthedocs.io/en/stable/reference/ImageChops.html#PIL.ImageChops.offset](https://pillow.readthedocs.io/en/stable/reference/ImageChops.html#PIL.ImageChops.offset)
- `PIL.Image.Image.transform`: [pillow.readthedocs.io/en/stable/reference/Image.html#PIL.Image.Image.transform](https://pillow.readthedocs.io/en/stable/reference/Image.html#PIL.Image.Image.transform)
- `skimage.transform.warp`: [scikit-image.org/docs/stable/api/skimage.transform.html#skimage.transform.warp](https://scikit-image.org/docs/stable/api/skimage.transform.html#skimage.transform.warp)


In [ ]:
def apply_geometric_transform(image: NDArray, transform_matrix: NDArray) -> NDArray:

    h, w = image.shape[:2]
    is_color = len(image.shape) == 3
    c = image.shape[2] if is_color else 1

    inv_matrix = np.linalg.inv(transform_matrix)

    # generate output pixel grid (x', y')
    x_out, y_out = np.meshgrid(np.arange(w), np.arange(h))
    ones = np.ones_like(x_out)
    coords_out = np.stack([x_out, y_out, ones], axis=-1)  # (H, W, 3)

    # compute input coordinates (x, y) via inverse transformation
    coords_in = coords_out @ inv_matrix.T

    # normalize
    x_in = coords_in[..., 0] / coords_in[..., 2]
    y_in = coords_in[..., 1] / coords_in[..., 2]

    # initialize output image
    transformed = np.zeros_like(image)

    # round coordinates to nearest integer
    x = np.round(x_in).astype(int)
    y = np.round(y_in).astype(int)

    # create valid pixel mask
    mask = (x >= 0) & (x < w) & (y >= 0) & (y < h)

    # assign pixels
    if is_color:
        for channel in range(c):
            transformed[..., channel][mask] = image[y[mask], x[mask], channel]
    else:
        transformed[mask] = image[y[mask], x[mask]]

    return transformed.astype(image.dtype)

## <a id='toc3_1_'></a>[Rigid Transformations (Isometric)](#toc0_)

- They preserve the **Euclidean distance** (straight-line distances) and **angles** **between points** in an image.
- They are called **rigid** because they **do not alter the shape or size** of objects, only their **position** and **orientation** in space.


### <a id='toc3_1_1_'></a>[Translation (Shifting)](#toc0_)

- Moves every point in the image by a fixed amount $(t_x,t_y)$.
- **Matrix Representation:**

$$
M = \begin{bmatrix}
1 & 0 & t_x \\
0 & 1 & t_y \\
0 & 0 & 1
\end{bmatrix}, \quad
M^{-1} = \begin{bmatrix}
1 & 0 & -t_x \\
0 & 1 & -t_y \\
0 & 0 & 1
\end{bmatrix}
$$


#### <a id='toc3_1_1_1_'></a>[Manual](#toc0_)


In [ ]:
im_1_shift_1 = apply_geometric_transform(image=im_1, transform_matrix=np.array([[1, 0, 60], [0, 1, 0], [0, 0, 1]]))
im_1_shift_2 = apply_geometric_transform(image=im_1, transform_matrix=np.array([[1, 0, 0], [0, 1, 60], [0, 0, 1]]))
im_1_shift_3 = apply_geometric_transform(image=im_1, transform_matrix=np.array([[1, 0, 60], [0, 1, 60], [0, 0, 1]]))

im_2_shift_1 = apply_geometric_transform(image=im_2, transform_matrix=np.array([[1, 0, 120], [0, 1, 0], [0, 0, 1]]))
im_2_shift_2 = apply_geometric_transform(image=im_2, transform_matrix=np.array([[1, 0, 0], [0, 1, 120], [0, 0, 1]]))
im_2_shift_3 = apply_geometric_transform(image=im_2, transform_matrix=np.array([[1, 0, 120], [0, 1, 120], [0, 0, 1]]))

# plot
fig, axs = plt.subplots(1, 6, figsize=(18, 4), layout="compressed")
images = [im_1_shift_1, im_1_shift_2, im_1_shift_3, im_2_shift_1, im_2_shift_2, im_2_shift_3]
titles = ["x:60", "y:60", "x:60, y:60 [bilinear]", "x:120", "y:120", "x:120, y:120 [bilinear]"]
cmaps = ["gray", "gray", "gray", None, None, None]
for ax, img, cm, title in zip(axs, images, cmaps, titles):
    ax.imshow(img, cmap=cm)
    ax.set_title(title)
    ax.axis("off")
plt.show()

#### <a id='toc3_1_1_2_'></a>[Using OpenCV](#toc0_)

- OpenCV simplifies the $3 \times 3$ mapping matrix $M$ by dropping the redundant third row ($[0, 0, 1]$)

$$
\begin{bmatrix}
x' \\ y'
\end{bmatrix}
= M_{2 \times 3} \cdot
\begin{bmatrix}
x \\ y \\ 1
\end{bmatrix}
$$


In [ ]:
im_1_shift_4 = cv2.warpAffine(
    im_1,
    np.array([[1, 0, 60], [0, 1, 0]], dtype=np.float32),
    dsize=(im_1.shape[1], im_1.shape[0]),
    borderMode=cv2.BORDER_CONSTANT,
)

im_1_shift_5 = cv2.warpAffine(
    im_1,
    np.array([[1, 0, 0], [0, 1, 60]], dtype=np.float32),
    dsize=(im_1.shape[1], im_1.shape[0]),
    borderMode=cv2.BORDER_CONSTANT,
)

im_1_shift_6 = cv2.warpAffine(
    im_1,
    np.array([[1, 0, 60], [0, 1, 60]], dtype=np.float32),
    dsize=(im_1.shape[1], im_1.shape[0]),
    borderMode=cv2.BORDER_REPLICATE,
)

im_2_shift_4 = cv2.warpAffine(
    im_2,
    np.array([[1, 0, 120], [0, 1, 0]], dtype=np.float32),
    dsize=(im_2.shape[1], im_2.shape[0]),
    borderMode=cv2.BORDER_CONSTANT,
)

im_2_shift_5 = cv2.warpAffine(
    im_2,
    np.array([[1, 0, 0], [0, 1, 120]], dtype=np.float32),
    dsize=(im_2.shape[1], im_2.shape[0]),
    borderMode=cv2.BORDER_CONSTANT,
)

im_2_shift_6 = cv2.warpAffine(
    im_2,
    np.array([[1, 0, 120], [0, 1, 120]], dtype=np.float32),
    dsize=(im_2.shape[1], im_2.shape[0]),
    borderMode=cv2.BORDER_REPLICATE,
)

# plot
fig, axs = plt.subplots(1, 6, figsize=(18, 4), layout="compressed")
images = [im_1_shift_4, im_1_shift_5, im_1_shift_6, im_2_shift_4, im_2_shift_5, im_2_shift_6]
titles = ["x:60", "y:60", "x:60, y:60 [bilinear]", "x:120", "y:120", "x:120, y:120 [bilinear]"]
cmaps = ["gray", "gray", "gray", None, None, None]
for ax, img, cm, title in zip(axs, images, cmaps, titles):
    ax.imshow(img, cmap=cm)
    ax.set_title(title)
    ax.axis("off")
plt.show()

#### <a id='toc3_1_1_3_'></a>[Using PIL](#toc0_)


In [ ]:
# method 1
im_1_shift_7 = ImageChops.offset(img_1, xoffset=60, yoffset=0)
im_1_shift_8 = ImageChops.offset(img_1, xoffset=0, yoffset=60)
im_1_shift_9 = ImageChops.offset(img_1, xoffset=60, yoffset=60)

# method 2: more control over transformation
im_2_shift_7 = img_2.transform(img_2.size, Image.Transform.AFFINE, (1, 0, -120, 0, 1, 0))
im_2_shift_8 = img_2.transform(img_2.size, Image.Transform.AFFINE, (1, 0, 0, 0, 1, -120))
im_2_shift_9 = img_2.transform(img_2.size, Image.Transform.AFFINE, (1, 0, -120, 0, 1, -120))

# plot
fig, axs = plt.subplots(1, 6, figsize=(18, 4), layout="compressed")
images = [im_1_shift_7, im_1_shift_8, im_1_shift_9, im_2_shift_7, im_2_shift_8, im_2_shift_9]
titles = ["x:60", "y:60", "x:60, y:60 [bilinear]", "x:120", "y:120", "x:120, y:120 [bilinear]"]
cmaps = ["gray", "gray", "gray", None, None, None]
for ax, img, cm, title in zip(axs, images, cmaps, titles):
    ax.imshow(img, cmap=cm)
    ax.set_title(title)
    ax.axis("off")
plt.show()

#### <a id='toc3_1_1_4_'></a>[Using scikit-image](#toc0_)


In [ ]:
im_1_shift_10 = ski.transform.warp(
    im_1,
    inverse_map=ski.transform.AffineTransform(translation=(60, 0)).inverse,
)
im_1_shift_11 = ski.transform.warp(
    im_1,
    inverse_map=ski.transform.AffineTransform(translation=(0, 60)).inverse,
)
im_1_shift_12 = ski.transform.warp(
    im_1,
    inverse_map=ski.transform.AffineTransform(translation=(60, 60)).inverse,
    mode="edge",
)
im_2_shift_10 = ski.transform.warp(
    im_2,
    inverse_map=ski.transform.AffineTransform(translation=(120, 0)).inverse,
)
im_2_shift_11 = ski.transform.warp(
    im_2,
    inverse_map=ski.transform.AffineTransform(translation=(0, 120)).inverse,
)
im_2_shift_12 = ski.transform.warp(
    im_2,
    inverse_map=ski.transform.AffineTransform(translation=(120, 120)).inverse,
    mode="edge",
)

# plot
fig, axs = plt.subplots(1, 6, figsize=(18, 4), layout="compressed")
images = [im_1_shift_10, im_1_shift_11, im_1_shift_12, im_2_shift_10, im_2_shift_11, im_2_shift_12]
titles = ["x:60", "y:60", "x:60, y:60 [bilinear]", "x:120", "y:120", "x:120, y:120 [bilinear]"]
cmaps = ["gray", "gray", "gray", None, None, None]
for ax, img, cm, title in zip(axs, images, cmaps, titles):
    ax.imshow(img, cmap=cm)
    ax.set_title(title)
    ax.axis("off")
plt.show()

### <a id='toc3_1_2_'></a>[Rotation](#toc0_)

- Rotates points around a fixed center $(c_x,c_y)$ by an angle $\theta$ (counterclockwise).
- **Matrix Representation:**

$$
M = \begin{bmatrix}
\cos\theta & -\sin\theta & c_x(1-\cos\theta) + c_y\sin\theta \\
\sin\theta & \cos\theta & c_y(1-\cos\theta) - c_x\sin\theta \\
0 & 0 & 1
\end{bmatrix}, \quad
M^{-1} = \begin{bmatrix}
\cos\theta & \sin\theta & c_x(1-\cos\theta) - c_y\sin\theta \\
-\sin\theta & \cos\theta & c_y(1-\cos\theta) + c_x\sin\theta \\
0 & 0 & 1
\end{bmatrix}
$$


In [ ]:
def rotation_matrix(angle_deg: float, image_shape: tuple[int, ...]) -> NDArray[np.float32]:
    h, w = image_shape[:2]
    cx = (w - 1) / 2.0
    cy = (h - 1) / 2.0
    theta = np.radians(angle_deg)

    cos_theta = np.cos(theta)
    sin_theta = np.sin(theta)

    tx = cx * (1 - cos_theta) + cy * sin_theta
    ty = cy * (1 - cos_theta) - cx * sin_theta

    return np.array([[cos_theta, -sin_theta, tx], [sin_theta, cos_theta, ty], [0, 0, 1]], dtype=np.float32)

#### <a id='toc3_1_2_1_'></a>[Using OpenCV](#toc0_)


In [ ]:
# method 1
im_1_rotate_1 = cv2.warpAffine(
    im_1,
    rotation_matrix(45, im_1.shape)[:2],
    dsize=(im_1.shape[1], im_1.shape[0]),
    borderMode=cv2.BORDER_CONSTANT,
)
im_1_rotate_2 = cv2.warpAffine(
    im_1,
    rotation_matrix(90, im_1.shape)[:2],
    dsize=(im_1.shape[1], im_1.shape[0]),
    borderMode=cv2.BORDER_CONSTANT,
)
im_1_rotate_3 = cv2.warpAffine(
    im_1,
    rotation_matrix(257, im_1.shape)[:2],
    dsize=(im_1.shape[1], im_1.shape[0]),
    borderMode=cv2.BORDER_CONSTANT,
)

# method 2
im_1_rotate_4 = cv2.rotate(im_1, rotateCode=cv2.ROTATE_90_CLOCKWISE)
im_1_rotate_5 = cv2.rotate(im_1, rotateCode=cv2.ROTATE_90_COUNTERCLOCKWISE)
im_1_rotate_6 = cv2.rotate(im_1, rotateCode=cv2.ROTATE_180)

# plot
fig, axs = plt.subplots(1, 6, figsize=(18, 4), layout="compressed")
images = [im_1_rotate_1, im_1_rotate_2, im_1_rotate_3, im_1_rotate_4, im_1_rotate_5, im_1_rotate_6]
titles = ["im_1_rotate_1", "im_1_rotate_2", "im_1_rotate_3", "im_1_rotate_4", "im_1_rotate_5", "im_1_rotate_6"]
for ax, img, title in zip(axs, images, titles):
    ax.imshow(img, cmap="gray")
    ax.set_title(title)
    ax.axis("off")
plt.show()

#### <a id='toc3_1_2_2_'></a>[Using PIL](#toc0_)


In [ ]:
# method 1
im_2_rotate_1 = img_2.transform(
    img_2.size,
    Image.Transform.AFFINE,
    rotation_matrix(45, im_2.shape).ravel()[:6],
    resample=Image.BILINEAR,
)
im_2_rotate_2 = img_2.transform(
    img_2.size,
    Image.Transform.AFFINE,
    rotation_matrix(90, im_2.shape).ravel()[:6],
    resample=Image.BILINEAR,
)
im_2_rotate_3 = img_2.transform(
    img_2.size,
    Image.Transform.AFFINE,
    rotation_matrix(257, im_2.shape).ravel()[:6],
    resample=Image.BILINEAR,
)

# method 2
im_2_rotate_4 = img_2.rotate(45, resample=Image.BILINEAR, expand=True)
im_2_rotate_5 = img_2.rotate(90, resample=Image.BILINEAR, expand=True)
im_2_rotate_6 = img_2.rotate(257, resample=Image.BILINEAR, expand=True)

# plot
fig, axs = plt.subplots(1, 6, figsize=(18, 4), layout="compressed")
images = [im_2_rotate_1, im_2_rotate_2, im_2_rotate_3, im_2_rotate_4, im_2_rotate_5, im_2_rotate_6]
titles = ["im_2_rotate_1", "im_2_rotate_2", "im_2_rotate_3", "im_2_rotate_4", "im_2_rotate_5", "im_2_rotate_6"]
for ax, img, title in zip(axs, images, titles):
    ax.imshow(img)
    ax.set_title(title)
    ax.axis("off")
plt.show()

### <a id='toc3_1_3_'></a>[Reflection (Flipping)](#toc0_)

- Mirrors the image over an axis (e.g., vertical/horizontal flip).
- **Matrix Representation:**
  - To reflect around a vertical line at $x = c_x$:
  $$
  M = M^{-1} = \begin{bmatrix}
  -1 & 0 & 2c_x \\
  0 & 1 & 0 \\
  0 & 0 & 1
  \end{bmatrix}
  $$

  - To reflect around a Horizontal line at $y = c_y$:
  $$
  M =  M^{-1} = \begin{bmatrix}
  1 & 0 & 0 \\
  0 & -1 & 2c_y \\
  0 & 0 & 1
  \end{bmatrix}
  $$


#### <a id='toc3_1_3_1_'></a>[Using OpenCV](#toc0_)


In [ ]:
im_1_flip_1 = cv2.warpAffine(
    im_1,
    np.array([[-1, 0, im_1.shape[1]], [0, 1, 0]], dtype=np.float32),
    dsize=(im_1.shape[1], im_1.shape[0]),
    borderMode=cv2.BORDER_CONSTANT,
)

im_1_flip_2 = cv2.warpAffine(
    im_1,
    np.array([[1, 0, 0], [0, -1, im_1.shape[0]]], dtype=np.float32),
    dsize=(im_1.shape[1], im_1.shape[0]),
    borderMode=cv2.BORDER_CONSTANT,
)

im_1_flip_3 = cv2.warpAffine(
    im_1,
    np.array([[-1, 0, im_1.shape[1]], [0, -1, im_1.shape[0]]], dtype=np.float32),
    dsize=(im_1.shape[1], im_1.shape[0]),
    borderMode=cv2.BORDER_REPLICATE,
)

im_2_flip_1 = cv2.warpAffine(
    im_2,
    np.array([[-1, 0, im_2.shape[1]], [0, 1, 0]], dtype=np.float32),
    dsize=(im_2.shape[1], im_2.shape[0]),
    borderMode=cv2.BORDER_CONSTANT,
)

im_2_flip_2 = cv2.warpAffine(
    im_2,
    np.array([[1, 0, 0], [0, -1, im_2.shape[0]]], dtype=np.float32),
    dsize=(im_2.shape[1], im_2.shape[0]),
    borderMode=cv2.BORDER_CONSTANT,
)

im_2_flip_3 = cv2.warpAffine(
    im_2,
    np.array([[-1, 0, im_2.shape[1]], [0, -1, im_2.shape[0]]], dtype=np.float32),
    dsize=(im_2.shape[1], im_2.shape[0]),
    borderMode=cv2.BORDER_REPLICATE,
)

# plot
fig, axs = plt.subplots(1, 6, figsize=(18, 4), layout="compressed")
images = [im_1_flip_1, im_1_flip_2, im_1_flip_3, im_2_flip_1, im_2_flip_2, im_2_flip_3]
titles = ["im_1_flip_1", "im_1_flip_2", "im_1_flip_3", "im_2_flip_1", "im_2_flip_2", "im_2_flip_3"]
cmaps = ["gray", "gray", "gray", None, None, None]
for ax, img, cm, title in zip(axs, images, cmaps, titles):
    ax.imshow(img, cmap=cm)
    ax.set_title(title)
    ax.axis("off")
plt.show()

## <a id='toc3_2_'></a>[Affine Transformations](#toc0_)

- Affine transformations are a generalization of rigid transformations.
- These transformations **preserve parallel lines** but not necessarily **distances** or **angles**.


### <a id='toc3_2_1_'></a>[Scaling](#toc0_)

- It changes the size of the object but preserves its shape if the scaling is uniform.
- **Matrix Representation:**

  $$
  M = \begin{bmatrix}
  S_x & 0 & 0 \\
  0 & S_y & 0 \\
  0 & 0 & 1
  \end{bmatrix}, \quad
  M^{-1} = \begin{bmatrix}
  \frac{1}{S_x} & 0 & 0 \\
  0 & \frac{1}{S_y} & 0 \\
  0 & 0 & 1
  \end{bmatrix}
  $$


#### <a id='toc3_2_1_1_'></a>[Using OpenCV](#toc0_)


In [ ]:
im_1_scale_1 = cv2.warpAffine(
    im_1, np.array([[1.5, 0, 0], [0, 0.5, 0]], dtype=np.float32), dsize=(im_1.shape[1], im_1.shape[0])
)

im_1_scale_2 = cv2.warpAffine(
    im_1, np.array([[1, 0, 0], [0, 1.5, 0]], dtype=np.float32), dsize=(im_1.shape[1], im_1.shape[0])
)

im_1_scale_3 = cv2.warpAffine(
    im_1, np.array([[1.5, 0, 0], [0, 1.5, 0]], dtype=np.float32), dsize=(im_1.shape[1], im_1.shape[0])
)

im_2_scale_1 = cv2.warpAffine(
    im_2, np.array([[1.5, 0, 0], [0, 0.5, 0]], dtype=np.float32), dsize=(im_2.shape[1], im_2.shape[0])
)

im_2_scale_2 = cv2.warpAffine(
    im_2, np.array([[1, 0, 0], [0, 1.5, 0]], dtype=np.float32), dsize=(im_2.shape[1], im_2.shape[0])
)

im_2_scale_3 = cv2.warpAffine(
    im_2, np.array([[1.5, 0, 0], [0, 1.5, 0]], dtype=np.float32), dsize=(im_2.shape[1], im_2.shape[0])
)

# plot
fig, axs = plt.subplots(1, 6, figsize=(18, 4), layout="compressed")
images = [im_1_scale_1, im_1_scale_2, im_1_scale_3, im_2_scale_1, im_2_scale_2, im_2_scale_3]
titles = ["im_1_scale_1", "im_1_scale_2", "im_1_scale_3", "im_2_scale_1", "im_2_scale_2", "im_2_scale_3"]
cmaps = ["gray", "gray", "gray", None, None, None]
for ax, img, cm, title in zip(axs, images, cmaps, titles):
    ax.imshow(img, cmap=cm)
    ax.set_title(title)
    ax.axis("off")
plt.show()

### <a id='toc3_2_2_'></a>[Shear](#toc0_)

- It changes the shape of the object but preserves its area.
- **Matrix Representation:**
  - Vertical Shear:
  $$
  M = \begin{bmatrix}
  1 & 0 & 0 \\
  h_y & 1 & 0 \\
  0 & 0 & 1
  \end{bmatrix}, \quad
  M^{-1} = \begin{bmatrix}
  1 & 0 & 0 \\
  -h_y & 1 & 0 \\
  0 & 0 & 1
  \end{bmatrix}
  $$

  - Horizontal Shear:
  $$
  M = \begin{bmatrix}
  1 & h_x & 0 \\
  0 & 1 & 0 \\
  0 & 0 & 1
  \end{bmatrix}, \quad
  M^{-1} = \begin{bmatrix}
  1 & -h_x & 0 \\
  0 & 1 & 0 \\
  0 & 0 & 1
  \end{bmatrix}
  $$


#### <a id='toc3_2_2_1_'></a>[Using OpenCV](#toc0_)


In [ ]:
im_1_shear_1 = cv2.warpAffine(
    im_1,
    np.array([[1, 0, 0], [0.2, 1, 0]], dtype=np.float32),
    dsize=(im_1.shape[1], im_1.shape[0]),
    borderMode=cv2.BORDER_CONSTANT,
)

im_1_shear_2 = cv2.warpAffine(
    im_1,
    np.array([[1, 0.2, 0], [0, 1, 0]], dtype=np.float32),
    dsize=(im_1.shape[1], im_1.shape[0]),
    borderMode=cv2.BORDER_CONSTANT,
)

im_1_shear_3 = cv2.warpAffine(
    im_1,
    np.array([[1, 0.4, 0], [0.4, 1, 0]], dtype=np.float32),
    dsize=(im_1.shape[1], im_1.shape[0]),
    borderMode=cv2.BORDER_REPLICATE,
)

im_2_shear_1 = cv2.warpAffine(
    im_2,
    np.array([[1, 0, 0], [0.2, 1, 0]], dtype=np.float32),
    dsize=(im_2.shape[1], im_2.shape[0]),
    borderMode=cv2.BORDER_CONSTANT,
)

im_2_shear_2 = cv2.warpAffine(
    im_2,
    np.array([[1, 0.2, 0], [0, 1, 0]], dtype=np.float32),
    dsize=(im_2.shape[1], im_2.shape[0]),
    borderMode=cv2.BORDER_CONSTANT,
)

im_2_shear_3 = cv2.warpAffine(
    im_2,
    np.array([[1, 0.4, 0], [0.4, 1, 0]], dtype=np.float32),
    dsize=(im_2.shape[1], im_2.shape[0]),
    borderMode=cv2.BORDER_REPLICATE,
)

# plot
fig, axs = plt.subplots(1, 6, figsize=(18, 4), layout="compressed")
images = [im_1_shear_1, im_1_shear_2, im_1_shear_3, im_2_shear_1, im_2_shear_2, im_2_shear_3]
titles = ["im_1_shear_1", "im_1_shear_2", "im_1_shear_3", "im_2_shear_1", "im_2_shear_2", "im_2_shear_3"]
cmaps = ["gray", "gray", "gray", None, None, None]
for ax, img, cm, title in zip(axs, images, cmaps, titles):
    ax.imshow(img, cmap=cm)
    ax.set_title(title)
    ax.axis("off")
plt.show()

## <a id='toc3_3_'></a>[Projective Transformations (Homography)](#toc0_)

- They preserve straight lines but do not preserve parallelism, distances, or angles.
- These transformations are used to model perspective changes, such as when viewing a scene from a different angle.
- They do not have a predefined matrix structure with fixed parameters.

  $$
  M = \begin{bmatrix}
  h_{11} & h_{12} & h_{13} \\
  h_{21} & h_{22} & h_{23} \\
  h_{31} & h_{32} & h_{33}
  \end{bmatrix}
  $$


### <a id='toc3_3_1_'></a>[Perspective](#toc0_)

**Examples:**
- Perspective Tilt
  $$
  M = \begin{bmatrix}
  1 & 0 & 0 \\
  0 & 1 & 0 \\
  0.001 & 0.001 & 1
  \end{bmatrix}, \quad
  M^{-1} = \begin{bmatrix}
  1 & 0 & 0 \\
  0 & 1 & 0 \\
  -0.001 & -0.001 & 1
  \end{bmatrix}
  $$
  
- Perspective Scaling
  $$
  M = \begin{bmatrix}
  0.8 & 0 & 0 \\
  0 & 0.8 & 0 \\
  0.0005 & 0.0005 & 1
  \end{bmatrix}, \quad
  M^{-1} = \begin{bmatrix}
  1.25 & 0 & 0 \\
  0 & 1.25 & 0 \\
  -0.000625 & -0.000625 & 1
  \end{bmatrix}
  $$
  
- Perspective Rotation
  $$
  M = \begin{bmatrix}
  \cos \theta & -\sin \theta & 0 \\
  \sin \theta & \cos \theta & 0 \\
  0.0005 & 0.0005 & 1
  \end{bmatrix}, \quad
  M^{-1} = \begin{bmatrix}
  \cos \theta & \sin \theta & 0 \\
  -\sin \theta & \cos \theta & 0 \\
  -0.0005 \cos \theta + 0.0005 \sin \theta & -0.0005 \sin \theta - 0.0005 \cos \theta & 1
  \end{bmatrix}
  $$
  
- Perspective Translation
  $$
  M = \begin{bmatrix}
  1 & 0 & 100 \\
  0 & 1 & 50 \\
  0.0005 & 0.0005 & 1
  \end{bmatrix}, \quad
  M^{-1} = \begin{bmatrix}
  1 & 0 & -100 \\
  0 & 1 & -50 \\
  -0.0005 & -0.0005 & 1
  \end{bmatrix}
  $$
  
- Warping for Image Stitching
  $$
  M = \begin{bmatrix}
  0.9 & -0.1 & 50 \\
  0.1 & 0.9 & 20 \\
  0.0002 & 0.0002 & 1
  \end{bmatrix}, \quad
  M^{-1} = \begin{bmatrix}
  0.9 & 0.1 & -50 \cdot 0.9 - 20 \cdot 0.1 \\
  -0.1 & 0.9 & -50 \cdot (-0.1) - 20 \cdot 0.9 \\
  0.0002 & 0.0002 & 0.9 \cdot 0.9 - (-0.1) \cdot 0.1
  \end{bmatrix}
  $$
  

#### <a id='toc3_3_1_1_'></a>[Using OpenCV](#toc0_)


In [ ]:
H_perspective_1 = np.array([[1, 0, 0], [0, 1, 0], [0.001, 0.001, 1]], dtype=np.float32)
H_perspective_2 = np.array([[1, 0, 0], [0, 1, 0], [0.002, 0.002, 1]], dtype=np.float32)
H_perspective_3 = np.array([[1, 0, 0], [0, 1, 0], [0.005, 0.005, 1]], dtype=np.float32)

im_1_perspective_1 = cv2.warpPerspective(
    im_1,
    H_perspective_1,
    dsize=(im_1.shape[1], im_1.shape[0]),
    borderMode=cv2.BORDER_CONSTANT,
)

im_1_perspective_2 = cv2.warpPerspective(
    im_1,
    H_perspective_2,
    dsize=(im_1.shape[1], im_1.shape[0]),
    borderMode=cv2.BORDER_CONSTANT,
)

im_1_perspective_3 = cv2.warpPerspective(
    im_1,
    H_perspective_3,
    dsize=(im_1.shape[1], im_1.shape[0]),
    borderMode=cv2.BORDER_REPLICATE,
)

im_2_perspective_1 = cv2.warpPerspective(
    im_2,
    H_perspective_1,
    dsize=(im_2.shape[1], im_2.shape[0]),
    borderMode=cv2.BORDER_CONSTANT,
)

im_2_perspective_2 = cv2.warpPerspective(
    im_2,
    H_perspective_2,
    dsize=(im_2.shape[1], im_2.shape[0]),
    borderMode=cv2.BORDER_CONSTANT,
)

im_2_perspective_3 = cv2.warpPerspective(
    im_2,
    H_perspective_3,
    dsize=(im_2.shape[1], im_2.shape[0]),
    borderMode=cv2.BORDER_REPLICATE,
)

# Plot
fig, axs = plt.subplots(1, 6, figsize=(18, 4), layout="compressed")
images = [
    im_1_perspective_1,
    im_1_perspective_2,
    im_1_perspective_3,
    im_2_perspective_1,
    im_2_perspective_2,
    im_2_perspective_3,
]
titles = [
    "im_1_perspective_1",
    "im_1_perspective_2",
    "im_1_perspective_3",
    "im_2_perspective_1",
    "im_2_perspective_2",
    "im_2_perspective_3",
]
cmaps = ["gray", "gray", "gray", None, None, None]
for ax, img, cm, title in zip(axs, images, cmaps, titles):
    ax.imshow(img, cmap=cm)
    ax.set_title(title)
    ax.axis("off")
plt.show()